In [11]:
import os
execfile("../script/utils.py")
eventsPath = os.environ["YAHOO_DATA"]
installsRawRdd = sc.textFile(eventsPath + "/installs.txt")
#splitedRdd = splitedRdd.map(parseContextData2)
#splitedRdd.take(1)
#splitedRdd = sc.parallelize(splitedRdd.take(2000))
installsRawRdd.take(5)
#(uid,[[train],[test]]

[u'001e6d8e-cbe7-4374-8c38-f37962a457e9\tair.com.buffalo_studios.bingorush2\t1420974077\t47.237461\t-122.530907\tTacoma\t6\t3\tApp_Installed\tinstall',
 u'009ef415-eb39-4f3e-8714-2b22a6490fbb\tcom.pandora.android\t1421228526\t37.353397\t-121.944977\tSanta Clara\t2\t1\tApp_Installed\tinstall',
 u'01fb3e32-5e7f-498d-aa51-aa8eabdb12ac\tcom.fractal360.go.launcherex.theme.gfl\t1421543961\t33.842270\t-84.211357\tTucker\t5\t20\tApp_Installed\tinstall',
 u'01fb3e32-5e7f-498d-aa51-aa8eabdb12ac\tcom.jb.gokeyboard.theme.el\t1421543917\t33.842270\t-84.211357\tTucker\t5\t20\tApp_Installed\tinstall',
 u'01fb3e32-5e7f-498d-aa51-aa8eabdb12ac\tcom.jb.emoji.gokeyboard\t1421544179\t33.84229\t-84.211235\tTucker\t5\t20\tApp_Installed\tinstall']

In [17]:
EventDataRow = namedtuple("EventDataRow",
                          ["userId", "itemId", "ts", "latitude", "longitude", "city", "day_of_week", "time_of_day",
                           "event_type"])
def parseRawData(line):
    lineSplit = line.split("\t")
    return EventDataRow(userId=lineSplit[0],
                        itemId=lineSplit[1],
                        ts=int(lineSplit[2]),
                        latitude=float(lineSplit[3]),
                        longitude=float(lineSplit[4]),
                        city=lineSplit[5],
                        day_of_week=int(lineSplit[6]),
                        time_of_day=int(lineSplit[7]),
                        event_type=lineSplit[-1],)
eventsRdd = installsRawRdd.map(parseRawData).cache()
eventsRdd.take(5)

[EventDataRow(userId=u'001e6d8e-cbe7-4374-8c38-f37962a457e9', itemId=u'air.com.buffalo_studios.bingorush2', ts=1420974077, latitude=47.237461, longitude=-122.530907, city=u'Tacoma', day_of_week=6, time_of_day=3, event_type=u'install'),
 EventDataRow(userId=u'009ef415-eb39-4f3e-8714-2b22a6490fbb', itemId=u'com.pandora.android', ts=1421228526, latitude=37.353397, longitude=-121.944977, city=u'Santa Clara', day_of_week=2, time_of_day=1, event_type=u'install'),
 EventDataRow(userId=u'01fb3e32-5e7f-498d-aa51-aa8eabdb12ac', itemId=u'com.fractal360.go.launcherex.theme.gfl', ts=1421543961, latitude=33.84227, longitude=-84.211357, city=u'Tucker', day_of_week=5, time_of_day=20, event_type=u'install'),
 EventDataRow(userId=u'01fb3e32-5e7f-498d-aa51-aa8eabdb12ac', itemId=u'com.jb.gokeyboard.theme.el', ts=1421543917, latitude=33.84227, longitude=-84.211357, city=u'Tucker', day_of_week=5, time_of_day=20, event_type=u'install'),
 EventDataRow(userId=u'01fb3e32-5e7f-498d-aa51-aa8eabdb12ac', itemId=u'c

In [19]:
eventsRdd = parseInstallsRawRdd.map(parseRawData).cache()
userIdConversionDictionary = eventsRdd.map(lambda x: x.userId).distinct().zipWithIndex().collectAsMap()
userIdConversionDictionaryBroadcast = sc.broadcast(userIdConversionDictionary)
itemIdConversionDictionary = eventsRdd.map(lambda x: x.itemId).distinct().zipWithIndex().collectAsMap()
itemIdConversionDictionaryBroadcast = sc.broadcast(itemIdConversionDictionary)
cityConversionDictionary = eventsRdd.map(lambda x: x.city).distinct().zipWithIndex().collectAsMap()
cityConversionDictionaryBroadcast = sc.broadcast(cityConversionDictionary)
eventsConvertedRdd = eventsRdd.map(lambda x: EventDataRow(
    userId=userIdConversionDictionaryBroadcast.value[x.userId],
    itemId=itemIdConversionDictionaryBroadcast.value[x.itemId],
    ts=x.ts,
    latitude=x.latitude,
    longitude=x.longitude,
    city=cityConversionDictionaryBroadcast.value[x.city],
    day_of_week=x.day_of_week,
    time_of_day=x.time_of_day,
     event_type=x.event_type
    ))

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 18.0 failed 1 times, most recent failure: Lost task 0.0 in stage 18.0 (TID 21, localhost): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/mert/spark-1.4.1-bin-hadoop2.6/python/lib/pyspark.zip/pyspark/worker.py", line 111, in main
    process()
  File "/home/mert/spark-1.4.1-bin-hadoop2.6/python/lib/pyspark.zip/pyspark/worker.py", line 106, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/home/mert/spark-1.4.1-bin-hadoop2.6/python/lib/pyspark.zip/pyspark/serializers.py", line 263, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "<ipython-input-17-98e6c669dbd2>", line 5, in parseRawData
AttributeError: 'EventDataRow' object has no attribute 'split'

	at org.apache.spark.api.python.PythonRDD$$anon$1

In [ ]:
    finalRDD = eventsConvertedRdd.map(lambda x: [
        x.userId, (
            x.itemId,
            x.ts,
            x.latitude,
            x.longitude,)
    ])

    groupData = map((lambda (x, y): (x, sorted(list(y), key=lambda a: a[1]))), sorted(finalRDD.groupByKey().collect()))

    def detectMovement(x):
        data = x[1]
        newData = [(data[0][0], data[0][1], data[0][2], data[0][3], 1)]
        for i in xrange(1, len(data)):
            event = data[i]
            distance = haversine(event[3], event[2], data[i - 1][3], data[i - 1][2]) * 1000  #in meters
            time_difference = event[1] - newData[i - 1][1]  #in seconds
            moving = 1  #not available
            if time_difference <= 300:  #if 2 consecutive events are more than 300 seconds away, the movement is not available
                velocity = distance / time_difference if time_difference > 0 else -1
                if velocity < 0:
                    moving = 1;  #not available
                elif velocity >= 0 and velocity <= 1:
                    moving = 2  #standing still
                elif velocity <= 2.4:
                    moving = 3  #walking spead
                else:
                    moving = 4  #faster
            newData.append((event[0], event[1], event[2], event[3], moving))
        return (x[0], newData)

    groupData = sc.parallelize(groupData).map(detectMovement).cache()

    def convertLocation(line):
        listGroup = line[1]
        workGroup = [x for x in listGroup if datetime.datetime.fromtimestamp(int(x[1])).hour >= 6 and
                     datetime.datetime.fromtimestamp(int(x[1])).hour <= 18]
        numNearLocation = []
        i = 0
        for x in workGroup:
            numNearLocation.append(0);
            for y in workGroup:
                if haversine(x[3], x[2], y[3], y[2]) < 0.1:
                    numNearLocation[i] = numNearLocation[i] + 1
            i = i + 1
        index_work = numNearLocation.index(max(numNearLocation))
        #     workGroup = [(x[0],x[1],x[2],x[3],1) if haversine(x[3], x[2], workGroup[index_work][3], workGroup[index_work][2]) < 0.1
        #                  else (x[0],x[1],x[2],x[3],0) for x in workGroup]
        #workGroup3 = [(x[0],x[1],0)  for x in workGroup if haversine(x[3], x[2], workGroup[index][3], workGroup[index][2]) >= 0.1]

        homeGroup = [x for x in listGroup if datetime.datetime.fromtimestamp(int(x[1])).hour < 6 or
                     datetime.datetime.fromtimestamp(int(x[1])).hour > 18]

        numNearLocation = []
        i = 0
        for x in homeGroup:
            numNearLocation.append(0);
            for y in homeGroup:
                if haversine(x[3], x[2], y[3], y[2]) < 0.1:
                    numNearLocation[i] = numNearLocation[i] + 1
            i = i + 1
        index_home = numNearLocation.index(max(numNearLocation))
        #     homeGroup = [(x[0],x[1],x[2],x[3],2) if haversine(x[3], x[2], homeGroup[index_home][3], homeGroup[index_home][2]) < 0.1
        #                  else (x[0],x[1],x[2],x[3],0) for x in homeGroup]

        listGroup = [
            (x[0], x[1], x[4], 1) if haversine(x[3], x[2], workGroup[index_work][3], workGroup[index_work][2]) < 0.01
            else(
                (x[0], x[1], x[4], 2) if haversine(x[3], x[2], homeGroup[index_home][3],
                                                   homeGroup[index_home][2]) < 0.01
                else (x[0], x[1], x[4], 3)
            )
            for x in listGroup]

        listGroup = [(x[0], x[2], x[3], 1) if datetime.datetime.fromtimestamp(int(x[1])).hour >= 8 and
                                              datetime.datetime.fromtimestamp(int(x[1])).hour <= 13
                     else(
            (x[0], x[2], x[3], 2) if datetime.datetime.fromtimestamp(int(x[1])).hour >= 13 and
                                     datetime.datetime.fromtimestamp(int(x[1])).hour <= 18
            else (x[0], x[2], x[3], 3)
        )
                     for x in listGroup]


        #time = datetime.datetime.fromtimestamp(int(line[1][0][1]))
        #line[1][1] = datetime.datetime.fromtimestamp(int("1284101485")).strftime('%Y-%m-%d %H:%M:%S')
        #return line[0],(workGroup+homeGroup)[:20],listGroup[:20]#,len(workGroup+homeGroup),len(workGroup),len(homeGroup)
        return line[0], listGroup

    return groupData.map(convertLocation)